# Part 2: DICOM

by Brad Genereaux, Medical Imaging Alliance Manager, NVIDIA

## Contents
* [DICOM Concepts](#dicom)
* [Tutorial: Creating DICOM](#tutorial1)
  * [Step 1: Import Libraries](#step1)
  * [Step 2: Create DICOM Stub](#step2)
  * [Step 3: Create Default Values](#step3)
  * [Step 4: Fill DICOM Tags](#step4)
  * [Step 5: Create Pixel Payload](#step5)
  * [Step 6: Write DICOM File](#step6)
  * [Step 7: Upload to Orthanc](#step7)
  * [Step 8: Check Orthanc for Study](#step8)
* [Resources](#resources)

## DICOM Concepts <a id="dicom"></a>

### DICOM is Imaging
Standard for handling, storing, printing, and transmitting information in medical imaging, and includes both a file structure and communication protocol
https://en.wikipedia.org/wiki/DICOM 

### What is DICOM?
* **D**igital **I**maging and **CO**mmunications in **M**edicine
* International standard (ISO 12052) for medical imaging and related information
* First standard in 1985, current standard formed in 1993
* One of the most widely deployed healthcare messaging standards in the world
  * 100,000s of modalities producing images, decades in production
  * ~ 2 trillion images annually (50% in the US)
  * ~ 450 Petabytes per year; doubles every 5 years
  * ~ 4.5 Exabytes globally total

### The DICOM Standard
* Administered and published by
  * National Electrical Manufacturers Association – NEMA
  * Medical Imaging Technology Alliance – MITA
* Intellectual property
  * DICOM trademark and copyright is held by NEMA
  * No license required to use the DICOM Standard in products
* http://dicom.nema.org
  * Download free electronic copies of the standard
    * All 20 parts are available in PDF, Word, HTML, and XML format
  * Plans and activities are publicly posted

### What does DICOM define?
* Formats for images, waveforms, derived structured data
  * Provides the quality and metadata necessary for clinical use
* Workflow management in the imaging department
* Media exchange and printing
* Service-based network protocols over TCP/IP and HTTP

* DICOM stores images
  * All kinds of images
  * CT, MR, X-Ray, Ultrasound, Angiography,PET, Ophthalmology, Documents, …
  * Single & Multi-frame; Volumes & Cines; B&W & Color; Original & Processed
* DICOM helps to manage images
  * Not just pixels → significant meta-data
  * Patient identification & demographics, theorder, acquisition, workflow context…
  * Can query / sort / auto-route / manage

### Active Modalities

| Modality | Modality | Modality | Modality |
|---|---|---|---|
| AR - Autorefraction             | HD - Hemodynamic Waveform                   | OPT - Ophthalmic Tomography             | RTSTRUCT - Radiotherapy Structure Set  |
| AU - Audio                      | IO - Intra-Oral Radiography                 | OPV - Ophthalmic Visual Field           | RWV - Real World Value Map             |
| BDUS - Bone Densitometry (US)   | IOL - Intraocular Lens Data                 | OSS - Optical Surface Scan              | SEG - Segmentation                     |
| BI - Biomagnetic imaging        | IVOCT - Intravascular Optical Coher Tomo    | OT - Other                              | SMR - Stereometric Relationship        |
| BMD - Bone Densitometry (X-Ray) | IVUS - Intravascular Ultrasound             | PLAN - Plan                             | SM - Slide Microscopy                  |
| CR - Computed Radiography       | KER - Keratometry                           | PR - Presentation State                 | SRF - Subjective Refraction            |
| CT - Computed Tomography        | KO - Key Object Selection                   | PT - Positron emission tomography (PET) | SR - SR Document                       |
| DG - Diaphanography             | LEN - Lensometry                            | PX - Panoramic X-Ray                    | STAIN - Automated Slide Stainer        |
| DOC - Document                  | LS - Laser surface scan                     | REG - Registration                      | TG - Thermography                      |
| DX - Digital Radiography        | MG - Mammography                            | RESP - Respiratory Waveform             | US - Ultrasound                        |
| ECG - Electrocardiography       | MR - Magnetic Resonance                     | RF - Radio Fluoroscopy                  | VA - Visual Acuity                     |
| EPS - Cardiac Electrophysiology | NM - Nuclear Medicine                       | RG - Radiographic imaging               | XA - X-Ray Angiography                 |
| ES - Endoscopy                  | OAM - Ophthalmic Axial Measurements         | RTDOSE - Radiotherapy Dose              | XC - External-camera Photography       |
| FID - Fiducials                 | OCT - Optical Coherence Tomography          | RTIMAGE - Radiotherapy Image            |                                        |
| GM - General Microscopy         | OPM - Ophthalmic Mapping                    | RTPLAN - Radiotherapy Plan              |                                        |
| HC - Hard Copy                  | OP - Ophthalmic Photography                 | RTRECORD - RT Treatment Record          |                                        |


### Other DICOM Components
* Store Non-Image Supportive Data
  * Fetal growth, cardiac output, tumor size, CAD findings, ECG Waveforms, …
* Manage (Imaging) Workflow
  * Modality Worklists, Progress updates,Storage Commitment
* Display Images
  * Screen calibration, annotations, layouts,key image flagging, …
* Distribute Images
  * Network push/pull,Media Transfer (CD, USB, Blu-ray…),Email Attachments,Web Protocols (!! DICOMweb !!)
* Store Analysis Results
  * Registrations, Segmentations,Implant Models, …
* Secure
  * Audit Trails, De-identification Schemes,Encryption


### Image Metadata

Each patient has **X** studies, which has **Y** series, which has **Z** instances, which may have **f** frames.

An instance, on disk, is represented as a self-contained file. Some imaging studies that have many slices, like a CT exam, in the early days, would be represented by as many files as slices, which ended up in a lot of repetitious data - and thus, the "multi-frame" instance was created.

Each instance has metadata at various levels - the following is a set of such metadata:

| Study           | Series        | Instance        | 
| ---             | ---           | ---             |
| Study UID       | Series UID    | Instance UID    |
| Date of Study   | Body Part     | SOP Class UID   |
| Description     | Modality      | Height (Rows)   |
| Refer Physician | Description   | Width (Columns) |
| Accession       | Series Number | Position        |

### Terminology
* Service + Object = **S**ervice **O**bject **P**air
 * “Image Storage” + “MRI” = “MRI Image Storage”
* SOP Class used to define a file type
* SCU = **S**ervice **C**lass **U**ser (“client”)
* SCP = **S**ervice **C**lass **P**rovider (“server”)
* AE = Reference to the **A**pplication **E**ntity (a client or a server)

Example SOP Classes (https://www.dicomlibrary.com/dicom/sop/)

| SOP Class UID | Description |
|---|---|
| 1.2.840.10008.5.1.4.1.1.1     | CR Image Storage                         |
| 1.2.840.10008.5.1.4.1.1.1.3   | Digital Intra – oral X-Ray Image Storage |
| 1.2.840.10008.5.1.4.1.1.2.1   | Enhanced CT Image Storage	               |
| 1.2.840.10008.5.1.4.1.1.20    | NM Image Storage                         |
| 1.2.840.10008.5.1.4.1.1.481.2 | Radiation Therapy Dose Storage	       |
| 1.2.840.10008.5.1.4.1.1.481.3 | Radiation Therapy Structure Set Storage  |
| 1.2.840.10008.5.1.4.1.1.88.11 | Basic Text SR                            |
| 1.2.840.10008.5.1.4.1.1.9.1.1 | 12-lead ECG Waveform Storage             |

### Metadata Tags
* DICOM’s binary structure for representing metadata; i.e.,
 * (0010,0010) -> Patient name
 * (0008,0090) -> Referring physician name
* Described in DICOM PS3.6
* Tags are defined using a VR (a data type) and an VM (multiplicity factor)
* See https://dicom.innolitics.com/ciods

### Unique Identification
* UIDs are unique identifiers of DICOM content
 * Study, series and SOP instances all have globally unique UIDs
 * Different than accession numbers, which relate to orders and are not guaranteed to be unique
* “Atomic” – DICOM files are not “edited”
 * You are not supposed to change DICOM objects themselves; if you do, they must get an updated SOP Instance UID (some exceptions in practice)

### Key Objects and Markup
* Key Objects
 * Way to represent a set of images besides “all of the images”
 * For example, “marked for review”, “marked for quality issues”, “XDS publishable set”
* GSPS
 * Markup and annotations on the images
 * Includes factors like window/level
 * Applied on images or image sets
 * Related methods include SEG (Segmentation) and CAD (Computer-aided detection) objects


### Reading The Standard
| Part | Part |
|---|---|
| Part 1: Introduction and Overview                            | Part 11: Media Storage Application Profiles                        |
| Part 2: Conformance                                          | Part 12: Media Formats and Physical Media for Media Interchange    |
| Part 3: Information Object Definitions                       | Part 14: Grayscale Standard Display Function                       |
| Part 4: Service Class Specifications                         | Part 15: Security and System Management Profiles                   |
| Part 5: Data Structures and Encoding                         | Part 16: Content Mapping Resource                                  |
| Part 6: Data Dictionary                                      | Part 17: Explanatory Information                                   |
| Part 7: Message Exchange                                     | Part 18: Web Services                                              |
| Part 8: Network Communication Support for Message Exchange   | Part 19: Application Hosting                                       |
| Part 10: Media Storage and File Format for Media Interchange | Part 20: Imaging Reports using HL7 Clinical Document Architecture  |

### Working Groups

| Group | Group |
|---|---|
| DSC – DICOM Standard Committee               | WG-17 3D                                             |
| WG-01 Cardiac and Vascular Information       | WG-18 Clinical Trials and Research                   |
| WG-02 Projection Radiography and Angiography | WG-19 Dermatology                                    |
| WG-03 Nuclear Medicine                       | WG-20 Integration of Imaging and Information Systems |
| WG-04 Compression                            | WG-21 Computed Tomography                            |
| WG-05 Exchange Media                         | WG-22 Dentistry                                      |
| WG-06 Base Standard                          | WG-23 Artificial Intelligence/Application Hosting    |
| WG-07 Radiotherapy                           | WG-24 DICOM in Surgery                               |
| WG-08 Structured Reporting and CDE           | WG-25 Veterinary Medicine                            |
| WG-09 Ophthalmology                          | WG-26 Pathology                                      |
| WG-10 Strategic Advisory                     | WG-27 Web Services for DICOM                         |
| WG-11 Display Function Standard              | WG-28 Physics                                        |
| WG-12 Ultrasound                             | WG-29 Education, Communication, and Outreach         |
| WG-13 Visible Light                          | WG-30 Small Animal Imaging                           |
| WG-14 Security                               | WG-31 Conformance                                    |
| WG-15 Mammography and CAD                    | WG-32 Neurophysiology Data                           |
| WG-16 Magnetic Resonance                     | WG-33 Data Archive and Management                    |


## Tutorial 1 <a id="tutorial1"></a>

We want to create a conformant DICOM File and visualize it within a PACS.

We will take a regular image - http://localhost:8888/notebooks/images/nvidia.jpg - and encode it as a DICOM object of IOD "VL Photographic Image". We will then C-STORE it against a PACS running in this Docker, and open it up in its viewer.

### Step 1: Import Libraries <a id="step1"></a>

First, we will import the necessary libraries in Python to succeed in this tutorial.

In [ ]:
import os
import tempfile
import datetime

import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid
from pydicom.sequence import Sequence
from PIL import Image

import requests

print("Libraries imported!")

### Step 2: Create DICOM Stub <a id="step2"></a>

In this step, we will create the DICOM envelope, and save the basic characteristics on how we will store the file.

In [ ]:
# Create SOP Instance UID
strInstanceUID = generate_uid(prefix=None)

# creating DICOM file
pathDicom = '/workshop/content/dicom'
fileDicom = tempfile.NamedTemporaryFile(dir=pathDicom, prefix=strInstanceUID, suffix='.dcm').name

# envelope metadata
strSopClass = '1.2.840.10008.5.1.4.1.1.77.1.4'

# Populate required values for base meta information
dsBaseMetadata = Dataset()
dsBaseMetadata.MediaStorageSOPClassUID = strSopClass
dsBaseMetadata.ImplementationClassUID = strSopClass
dsBaseMetadata.MediaStorageSOPInstanceUID = strInstanceUID
# Populate required values for meta information
dsMetadata = FileDataset(fileDicom, {}, file_meta=dsBaseMetadata, preamble=b"\0" * 128)

print("Done building the envelope!")

### Step 3: Create Default Values <a id="step3"></a>

In this step, we will store some default values in variables that we will use in creating DICOM.

In [ ]:
# Pre-calculate some raw information for file
currentDateRaw = datetime.datetime.now()
currentDate = currentDateRaw.strftime('%Y%m%d')
currentTime = currentDateRaw.strftime('%H%M%S.%f')  # long format with micro seconds
strStudyUID = generate_uid(prefix=None);
strSeriesUID = generate_uid(prefix=None)
strPatientName = 'Test^PatientName'
strPatientID = '141262'
strPatientGender = 'M'
datBirth = currentDateRaw.strftime('%Y%m%d')
strReferringPhysician = 'SAMPLE^CLARA'
strAccession = '123456'

print("Done setting dataset values!")

### Step 4: Fill DICOM Tags <a id="step4"></a>

In this step, we will add the necessary required metadata tags for our SOP Class. In our example, we will be creating a VL Photographic Image. See https://dicom.innolitics.com/ciods/vl-photographic-image for a great way to browse through all of the tags we need to populate in this step.

Note that we are using DICOM hexadecimal tags; PyDicom does have short-hand attribute names that we could use. Also, note the use of sequences (and the sub-datasets that are populated within).

In [ ]:
# Patient and Order details
dsMetadata.PatientName = strPatientName
dsMetadata.add_new(0x00100020, 'LO', strPatientID) # patient ID
dsMetadata.add_new(0x00100030, 'DA', datBirth) # Patient DOB
dsMetadata.add_new(0x00100040, 'CS', strPatientGender) # patient gender
dsMetadata.add_new(0x00080020, 'DA', currentDate) # Study Date (0008,0020) Date the Study started. (Must be created but can be left blank if unknown)
dsMetadata.add_new(0x00080030, 'TM', currentTime) # Study Time (0008,0030) Time the Study started. (Must be created but can be left blank if unknown)
dsMetadata.add_new(0x00080090, 'PN', strReferringPhysician) # Referring Physician’s Name (0008,0090) Name of the patient’s referring physician (Must be created but can be left blank if unknown)
dsMetadata.add_new(0x00200010, 'SH', strAccession) # Study ID (0020,0010) User or equipment generated Study identifier. (Must be created but can be left blank if unknown)
dsMetadata.add_new(0x00080050, 'SH', strAccession) # Accession Number (0008,0050) A RIS generated number that identifies the order for the Study. (Must be created but can be left blank if unknown)
# Study Details
dsMetadata.add_new(0x0020000D, 'UI', strStudyUID) # Study Instance UID (0020,000D) Unique identifier for the Study.
dsMetadata.add_new(0x00080060, 'CS', 'XC') # Modality (0008,0060) Type of equipment that originally acquired the data used to create the images in this Series.
# Series Details
dsMetadata.add_new(0x0020000E, 'UI', strSeriesUID) # Series Instance UID (0020,000E) Unique identifier of the Series.
dsMetadata.add_new(0x00200011, 'IS', '1') # Series Number (0020,0011) A number that identifies this Series. (Must be created but can be left blank if unknown)
# Instance Details
dsMetadata.add_new(0x00080016, 'UI', strSopClass) # SOP Class UID (0008,0016) Uniquely identifies the SOP Class.
dsMetadata.add_new(0x00080018, 'UI', strInstanceUID) # SOP Instance UID (0008,0018) Uniquely identifies the SOP Instance.
dsMetadata.add_new(0x00200013, 'IS', 1) # Instance Number (0020,0013) A number that identifies this image.(Must be created but can be left blank if unknown)
dsMetadata.add_new(0x00200020, 'CS', 'A') # Patient Orientation (0020,0020) Patient direction of the rows and columns of the image. Required if image does not require Image Orientation
dsMetadata.add_new(0x00080070, 'LO', 'NVIDIA')  # Manufacturer
dsMetadata.add_new(0x00081090, 'LO', 'CLARA')  # Manufacturer Model
dsMetadata.add_new(0x00181000, 'LO', '0000')  # Serial Number
dsMetadata.add_new(0x00181020, 'LO', '1')  # Software Version Number
dsMetadata.ContentDate = currentDate
dsMetadata.ContentTime = currentTime
dsMetadata.add_new(0x00200060, 'CS', 'L')  # Laterality
dsMetadata.add_new(0x00200020, 'CS', 'A\P')  # Patient Orientation
# set acquisition context
seqAcquisitionContext = Sequence()
dsAcquisitionContext = Dataset()
## create SegmentedPropertyCategoryCodeSequence
seqConceptNameCode = Sequence()
dsConceptNameCode = Dataset()
dsConceptNameCode.add_new(0x00080100, 'SH', '121322')  # Code Value
dsConceptNameCode.add_new(0x00080102, 'SH', 'DCM')  # Coding Scheme Validator
dsConceptNameCode.add_new(0x00080104, 'LO', 'Source image for image processing operation')  # Code Meaning
seqConceptNameCode.append(dsConceptNameCode)
dsAcquisitionContext.add_new(0x0040A043, 'SQ', seqConceptNameCode)
## other fields in acquisition context
dsAcquisitionContext.add_new(0x0040A160, 'UT', 'N/A')  # Text
seqAcquisitionContext.append(dsAcquisitionContext)
dsMetadata.add_new(0x00400555, 'SQ', seqAcquisitionContext)  # Shared Functional Groups sequence

### Step 5: Create Pixel Payload <a id="step5"></a>

In this step, we will set some image pixel metadata and then encode the pixels from the JPG file. 

In [ ]:
# Set pixel data information
dsMetadata.is_little_endian = True
dsMetadata.is_implicit_VR = True
dsMetadata.add_new(0x00280100, 'US', 8)  # Bits allocated
dsMetadata.BitsStored = 8
dsMetadata.HighBit = 7
dsMetadata.PixelRepresentation = 0
dsMetadata.SamplesPerPixel = 3
dsMetadata.add_new(0x00280004, 'CS', 'RGB') # (0028, 0004): Photometric Interpretation
dsMetadata.add_new(0x00280006, 'US', 0) # (0028, 0006): Planar Configuration
dsMetadata.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
dsMetadata.add_new(0x00282110, 'CS', '01') # (0028, 2110): Compression Flag
dsMetadata.ImageType = 'ORIGINAL\PRIMARY'
# load pixels
fileImg = Image.open(requests.get("http://localhost:8888/notebooks/images/nvidia.jpg", stream=True).raw)

dsMetadata.PixelData = fileImg.tobytes()
dsMetadata.add_new(0x00280010, 'US',  fileImg.size[1] ) # (0028, 0010): Rows
dsMetadata.add_new(0x00280011, 'US',  fileImg.size[0] ) # (0028, 0011): Columns
arrPixels = dsMetadata.pixel_array
arrPixels = arrPixels.reshape(arrPixels.shape[0],arrPixels.shape[1],arrPixels.shape[2])
arrPixels = arrPixels.transpose(0,1,2)
dsMetadata.PixelData = arrPixels.tostring()

### Step 6: Write DICOM File <a id="step6"></a>

In this step, we will write the file to disk. We will then examine the resulting values, and then optionally download the DICOM file for disk-based viewing.

In [ ]:
# output file
print('Writing test file', fileDicom)
dsMetadata.save_as(fileDicom, False)
print('File saved.')

# print out metadata
print('Printing headers')
print(dsMetadata)

# download
rightSide = len(fileDicom) - len(pathDicom) - 1
print(rightSide)
print('Download DICOM file: http://localhost:8888/notebooks/dicom/' + fileDicom[-rightSide:] )

#### Verify DICOM object
We can then verify the DICOM file using David Clunie's dicom3tools library tool called "dciodvfy". If we only see the DICOM SOP Class type with no errors, we should have a DICOM-compliant object.

In [ ]:
print('Verifying ', fileDicom)
os.environ["DICOM_FILE_TO_VERIFY"] = fileDicom
!dciodvfy $DICOM_FILE_TO_VERIFY

### Step 7: Upload to Orthanc <a id="step7"></a>

In this step, we will store this DICOM object to the Orthanc instance running within this Docker container. The DICOM port is 4242, and we will use Implicit Little Endian transfer syntax.

In [ ]:
from pydicom import dcmread

from pynetdicom import AE
from pynetdicom.sop_class import CTImageStorage
from pynetdicom.sop_class import VerificationSOPClass
from pynetdicom.sop_class import VLPhotographicImageStorage

# Initialise the Application Entity
ae = AE()

# Add a requested presentation context
ae.add_requested_context(VLPhotographicImageStorage)

# Associate with peer AE at IP 127.0.0.1 and port 4242
assoc = ae.associate('127.0.0.1', 4242)

# Send DICOM
status = assoc.send_c_store(dsMetadata)

# Release the association
assoc.release()

print('C-STORE request status: 0x{0:04x} (0x0000 represents success)'.format(status.Status))

### Step 8: Check Orthanc for Study<a id="step8"></a>

Open http://localhost:8042 to look for the study you have just uploaded.

## Resources <a id="resources"></a>
* Viewer: MicroDICOM (http://www.microdicom.com/)
* Python, PyDICOM
* A JPEG file to encode
* IOD Definition
 * DICOM Standard: https://www.dicomstandard.org/current/
 * Innolitics website: https://dicom.innolitics.com/ciods/vl-photographic-image
* DCIODVFY
 * Command-line validation tool: https://www.dclunie.com/dicom3tools/dciodvfy.html

